In [1]:
#a good example is provided here
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
from torch.autograd import Variable
from PIL import Image
from os import listdir
import json
import numpy as np
from random import shuffle

# globals 
disp = False

def loadImages(path):
    # return array of images

    imagesList = listdir(path)
    loadedImages = []
    for image in imagesList:
        img = Image.open(path + image)
        loadedImages.append(img)

    return loadedImages

#define prepro transform
'''
from:
https://github.com/pytorch/examples/blob/409a7262dcfa7906a92aeac25ee7d413baa88b67/imagenet/main.py#L108-L113
https://github.com/pytorch/examples/blob/409a7262dcfa7906a92aeac25ee7d413baa88b67/imagenet/main.py#L94-L95
'''
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
#    normalize
])

#provide path to true and fake images
true_path= "raw/true/"
fake_path= "raw/negative/"

# load images in the dir
true_images = loadImages(true_path)
fake_images = loadImages(fake_path)

#be sure the images are rightly loaded
if disp:
    true_images[0].show()
    fake_images[0].show()

# Now preprocess the image

#define tensors to hold the images in memory
fake_tensors, true_tensors = [], []

for imgs in true_images:
    true_temp = preprocess(imgs)
    true_temp = Variable(true_temp)
    true_tensors.append(true_temp)

for imgs in fake_images:
    fake_temp = preprocess(imgs)
    fake_temp = fake_temp.view(-1, 3,)
    fake_temp = Variable(fake_temp)
    fake_tensors.append(fake_temp)

#shuffle the images
shuffle(true_images)
shuffle(fake_images)
# fakenreal_tensors = fake_tensors + true_tensors

# print(fakenreal_tensors[240])
 
# #be sure the images are properly loaded in memory
# print("Total # of true tensors: ", len(true_tensors), "true images size: ", true_tensors[0].size())
# print("Total # of fake tensors: ", len(fake_tensors), "fake images size: ", fake_tensors[0].size())
# print("True Var Size {}".format(true_tensor_variables[0].size()))
# print("Fake Var Size {}".format(fake_tensor_variables[0].size()))

#load labels file
labels_file = open('labels.json').read()
labels = np.fromiter(json.loads(labels_file),int)

#take labels to tensor
labels = torch.from_numpy(labels)
labels_var = Variable(labels)
# labels = labels_var.unsqueeze(1).expand(99, 2,1)

if disp:
    print(labels)  

In [11]:
#Now separate true and fake to training and testing sets
#we'll do 80:20 ratio
X_tr = int(0.8*len(true_tensors))
X_te = len(true_tensors) - X_tr

# allocate tensors memory
train_X = torch.LongTensor(X_tr, true_tensors[0].size(0), true_tensors[0].size(1),
                          true_tensors[0].size(2))
train_Y = torch.LongTensor(train_X.size(0), 1)

#create testing set
test_X = torch.LongTensor(X_te, true_tensors[0].size(0), true_tensors[0].size(1),
                          true_tensors[0].size(2))
test_Y = torch.LongTensor(test_X.size(0), 2)

#Now copy tensors over 
train_X = torch.stack(true_tensors[:X_tr], 0)
train_Y = Variable(torch.stack(labels.unsqueeze(0).expand(int(X_tr/2), 2), 0).view(-1, X_tr).t())

#testing set
test_X = torch.stack(true_tensors[(X_tr):], 0)
test_Y = Variable(torch.stack(labels.unsqueeze(0).expand(int(X_te/2), 2), 0).view(-1, X_te).t())
                                      
#check size of slices
print('train_X and train_Y sizes: {} | {}'.format(train_X.size(), train_Y.size()))
print('test_X and test_Y sizes: {} | {}'.format(test_X.size(), test_Y.size()))

train_X and train_Y sizes: torch.Size([80, 3, 224, 224]) | torch.Size([80, 1])
test_X and test_Y sizes: torch.Size([20, 3, 224, 224]) | torch.Size([20, 1])


In [15]:
# Now build the CNN module
# CNN Model (2 conv layer)
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        '''
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
            padding=0, dilation=1, groups=1, bias=True)

            in_channels (int) – Number of channels in the input image
            out_channels (int) – Number of channels produced by the convolution
            kernel_size (int or tuple) – Size of the convolving kernel
            stride (int or tuple, optional) – Stride of the convolution
            padding (int or tuple, optional) – Zero-padding added to both sides of the input
            dilation (int or tuple, optional) – Spacing between kernel elements
            groups (int, optional) – Number of blocked connections from input channels to output channels
            bias (bool, optional) – If True, adds a learnable bias to the output
        '''
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [59]:
import os
from os import listdir
from PIL import Image

pwd = os.getcwd()
print((pwd))

face_pos = '/raw/face_pos'
face_neg = '/raw/face_neg'

face_neg_1 = pwd + face_neg + '/' + 'neg_1'
face_neg_2 = pwd + face_neg + '/' + 'neg_2'
face_neg_3 = pwd + face_neg + '/' + 'neg_3'
face_neg_4 = pwd + face_neg + '/' + 'neg_4'

negative_list = []
neg_dirs = [face_neg_1, face_neg_2, face_neg_3, face_neg_4]
for dirs in neg_dirs:
    print((dirs))
    for img_path in listdir(dirs):
        base, ext = os.path.splitext(img_path)
        if ext == '.jpg':
            img = Image.open(dirs + '/' + img_path)
            negative_list.append(img)
            print('appending {} to {} list'.format(img_path, 'negative'))
            

/home/lex/catkin_ws/src/sensors/ensenso_detect/manikin
/home/lex/catkin_ws/src/sensors/ensenso_detect/manikin/raw/face_neg/neg_1
appending fake1_18.jpg to negative list
appending fake1_8.jpg to negative list
appending fake1_28.jpg to negative list
appending fake1_24.jpg to negative list
appending fake1_26.jpg to negative list
appending fake1_7.jpg to negative list
appending fake1_34.jpg to negative list
appending fake1_12.jpg to negative list
appending fake1_30.jpg to negative list
appending fake1_6.jpg to negative list
appending fake1_39.jpg to negative list
appending fake1_13.jpg to negative list
appending fake1_14.jpg to negative list
appending fake1_31.jpg to negative list
appending fake1_16.jpg to negative list
appending fake1_23.jpg to negative list
appending fake1_32.jpg to negative list
appending fake1_5.jpg to negative list
appending fake1_33.jpg to negative list
appending fake1_2.jpg to negative list
appending fake1_38.jpg to negative list
appending fake1_4.jpg to negative li

In [61]:
import torch
q = torch.Tensor((2, 3, 4))

ImportError: No module named torch